In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import ast
#pd.set_option('display.max_colwidth', -1)
import ast
from pandas.io.json import json_normalize
import plotly.graph_objs as go
import plotly.plotly as py
import cufflinks
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='solar')
# Any results you write to the current directory are saved as output.

## This notebook is based on identifying the crew members and the data munging techniques that is required for getting the cast and crew details. Ultimately some insights have been gathered based on networkx and plotly.

## I will update the notebook with more analysis and comments at a later point of time. But please do feel free to upvote or comment or any constructive criticisms.

In [ ]:
df_credits=pd.read_csv('../input/tmdb_5000_credits.csv')
df_movies=pd.read_csv('../input/tmdb_5000_movies.csv')

In [ ]:
df_credits.head()

In [ ]:
df_movies.head()

In [ ]:
from pandas.io.json import json_normalize

In [ ]:
print("length of the first json array from crew column ",len(df_credits['crew'].apply(lambda x: ast.literal_eval(x))[0]))

In [ ]:
print("length of the first json array from cast column ",len(df_credits['cast'].apply(lambda x: ast.literal_eval(x))[0]))

### Contents of first json array from crew column 

In [ ]:
df_credits['crew'].apply(lambda x: ast.literal_eval(x))[0]

### Contents of first json array from cast column

In [ ]:
df_credits['cast'].apply(lambda x: ast.literal_eval(x))[0]

### checking the keys of json array

In [ ]:
df_credits['crew'].apply(lambda x: ast.literal_eval(x))[0][0].keys()

In [ ]:
df_credits['cast'].apply(lambda x: ast.literal_eval(x))[0][0].keys()

### functions to convert json array to multiple columns

In [ ]:
def list_of_dicts(ld):
    mov_dict = dict()

    for d in ast.literal_eval(ld):
        if list(d.values())[4] in mov_dict:
            mov_dict[list(d.values())[4]].append(list(d.values())[5])
        else:
            mov_dict[list(d.values())[4]] = [list(d.values())[5]]
    return mov_dict

In [ ]:
def list_of_dicts_cast(ld):
    mov_dict = dict()

    for d in ast.literal_eval(ld):
        if list(d.values())[6] in mov_dict:
            mov_dict[list(d.values())[6]].append(list(d.values())[5])
        else:
            mov_dict[list(d.values())[6]] = [list(d.values())[5]]
    return mov_dict

In [ ]:
df_A = json_normalize(df_credits['crew'].apply(list_of_dicts).tolist())#.add_prefix('job.')
df_B = json_normalize(df_credits['cast'].apply(list_of_dicts_cast).tolist())

In [ ]:
df_credits_crew = df_credits[['movie_id', 'title']].join([df_A])

In [ ]:
df_credits_cast = df_credits[['movie_id', 'title']].join([df_B])

In [ ]:
df_credits_crew_1 = pd.melt(df_credits_crew, id_vars=['movie_id','title'], var_name='Job').dropna()

In [ ]:
df_credits_crew_1.head()

In [ ]:
df_credits_cast_1 = pd.melt(df_credits_cast, id_vars=['movie_id','title'], var_name='Order').dropna()

In [ ]:
df_credits_crew_2 = df_credits_crew_1.value.apply(pd.Series) \
    .merge(df_credits_crew_1, right_index = True, left_index = True) \
    .drop(columns="value", axis = 1) \
    .melt(id_vars = ['movie_id', 'title','Job'], value_name = "name") \
    .drop("variable", axis = 1) \
    .dropna()

In [ ]:
df_credits_cast_2 = df_credits_cast_1.value.apply(pd.Series) \
    .merge(df_credits_cast_1, right_index = True, left_index = True) \
    .drop(columns="value", axis = 1) \
    .melt(id_vars = ['movie_id', 'title','Order'], value_name = "name") \
    .drop("variable", axis = 1) \
    .dropna()

In [ ]:
df_credits_cast_2[df_credits_cast_2['title'] == 'Avatar'].head(15)

In [ ]:
df_credits_cast_2.head(10)

In [ ]:
df_credits_crew_2[df_credits_crew_2['title'] == 'Avatar'].head(10)

In [ ]:
df_credits_crew_3 = df_credits_crew_2.merge(df_movies).drop(columns=['genres','keywords','production_companies','id','original_title','production_countries','spoken_languages'],axis=1)

In [ ]:
df_credits_cast_3 = df_credits_cast_2.merge(df_movies).drop(columns=['genres','keywords','production_companies','id','original_title','production_countries','spoken_languages'],axis=1)

In [ ]:
df_credits_cast_3.head(10)

In [ ]:
df_credits_crew_3.head()

In [ ]:
df_movies_1 = df_movies.drop(columns=['genres','keywords','production_companies','id','production_countries','spoken_languages'],axis=1)

In [ ]:
df_movies_1.head()

In [ ]:
df_movies_1['release_date'] =  pd.to_datetime(df_movies_1['release_date'], format='%Y-%m-%d')

### Histogram of various metrics to measure movie's success

In [ ]:
df_movies_1['vote_average'].iplot(
    kind='hist',
    bins=30,
    xTitle='vote',
    linecolor='black',
    yTitle='count',
    title='Vote Distribution')

In [ ]:
df_movies_1['popularity'].iplot(
    kind='hist',
    bins=30,
    xTitle='popularity',
    linecolor='black',
    yTitle='count',
    title='Popularity Distribution')

In [ ]:
df_movies_2 = df_movies_1.set_index('release_date')

In [ ]:
df_movies_2.iplot(
    y = 'revenue',
    mode='markers',
    opacity=0.8,
    size=8,
    symbol=1,
    text='original_title',
    xTitle='Date',
    yTitle='Revenue',
    title='Revenue Against Time')

In [ ]:
df_movies_2.iplot(
    y='runtime',
    mode='markers',
    opacity=0.8,
    size=8,
    symbol=1,
    text='original_title',
    xTitle='Date',
    yTitle='Runtime',
    title='Runtime Against Time')

In [ ]:
df_movies_2.iplot(
    x='runtime',
    y='revenue',
    xTitle='Run Time',
    yTitle='Revenue',
    text='original_title',
    mode='markers',
    title='Runtime vs Revenue')

In [ ]:
df_movies_2.iplot(
    x='popularity',
    y='revenue',
    xTitle='popularity',
    yTitle='Revenue',
    text='original_title',
    mode='markers',
    title='Popularity vs Revenue')

In [ ]:
import plotly.figure_factory as ff

figure = ff.create_scatterplotmatrix(
    df_movies_2.select_dtypes(exclude='object'),
    height=1000,
    width=1200,
    text=df_movies_2['original_title'],
    diag='histogram')
iplot(figure)

In [ ]:
import networkx as nx 
import matplotlib.pyplot as plt

In [ ]:
df_credits_cast_3.head(2)

### Creating a network of Movies acted by famous actors

In [ ]:
def actor_network(df,name):
    df  =  df[df["name"] == name][["Order","title"]]
    plt.figure(figsize=(30,32))
    G   = nx.from_pandas_edgelist(df,"Order","title",create_using=nx.Graph())
    nx.draw_kamada_kawai(G,with_labels = True,
                         node_size  = 3500,
                         node_color = 'green',
                         node_shape = "h",
                         edgecolor  = 'blue',
                         width=5,
                         linewidths  = 5 ,
                         font_size  = 15 ,
                         alpha=.8)
    
    plt.title("Movies acted by " + str(name) , fontsize = 25 , color = "red")
    

### This network graph allows us to visualize the titles acted by the famous actor before he/she became famous

In [ ]:
actor_network(df_credits_cast_3,'Sam Worthington')

### Being a fan of Sam Worthington, i have not known Hart's War before visualizing this graph. 

In [ ]:
actor_network(df_credits_cast_3,'Zoe Saldana')

## Here the nodes 0,1,2.. represent the order of their importance in the film with 0 being the most important or primary actor. This graph allowed us to know movie titles acted by Sam Worthington where he was not popular. Hart's War is one among them.

In [ ]:
def title_network(df,name):
    df  =  df[df["title"] == name][["name","title"]]
    plt.figure(figsize=(30,32))
    G   = nx.from_pandas_edgelist(df,"name","title",create_using=nx.Graph())
    nx.draw_kamada_kawai(G,with_labels = True,
                         node_size  = 3500,
                         node_color = 'turquoise',
                         node_shape = "h",
                         edgecolor  = 'blue',
                         width=5,
                         linewidths  = 5 ,
                         font_size  = 15 ,
                         alpha=.8)
    
    plt.title("Entire cast of " + str(name) , fontsize =25 , color = "blue")
    

In [ ]:
title_network(df_credits_cast_3,"Avatar")

In [ ]:
title_network(df_credits_cast_3,"Hart's War")

In [ ]:
df_credits_crew_2.head()

In [ ]:
top_crew_members = pd.DataFrame(df_credits_crew_2.groupby('name')['title'].count().sort_values(ascending=False).reset_index()[:15])

In [ ]:
top_crew_members.name.tolist()

In [ ]:
top_crew_members_1 = df_credits_crew_2[df_credits_crew_2['name'].isin(top_crew_members.name.tolist())]

In [ ]:
top_crew_members_1.head()

In [ ]:
def crew_network(df,name):    
    df  =  df[df["name"] == name][["Job","title"]]
    plt.figure(figsize=(30,32))
    G   = nx.from_pandas_edgelist(df,"Job","title",create_using=nx.Graph())
    nx.draw_kamada_kawai(G,#dist=df.to_dict(),
                         with_labels = True,
                         node_size  = 3500,
                         node_color = 'grey',
                         node_shape = "h",
                         edgecolor  = 'blue',
                         width=5,
                         linewidths  = 5 ,
                         font_size  = 15 ,
                         alpha=.8)
    
    plt.title("Movies worked by " + str(name) , fontsize =26 , color = "red")

### This graph allowed us to visualize the different roles and movie titles where Robert Rodriguez and Steven Spielberg had worked. 

In [ ]:
crew_network(top_crew_members_1,"Robert Rodriguez")

In [ ]:
crew_network(top_crew_members_1,"Steven Spielberg")

In [ ]:
crew_network(top_crew_members_1,"Hans Zimmer")